In [5]:
from result_saver import SaverProvider
from Scratch import metadata_loader
from Scratch import find_closest_calib_jobs

import numpy as np
from tqdm import tqdm
from time import sleep
from datetime import datetime
import json
import os

from soft_info import get_noise_dict_from_backend, get_avgs_from_dict, get_repcode_IQ_map
from soft_info import RepetitionCodeStimCircuit, inv_qubit_mapping, gaussianIQConvertor
from soft_info import get_cols_to_keep, generate_subsets_with_center, get_subsample_layout
from Scratch import load_calibration_memory
import cpp_soft_info as csi

def decode(DEVICE, LOGICAL, XBASIS, ROUNDS, file_name):
    provider = SaverProvider()

    state = "X" if XBASIS else "Z"
    state += LOGICAL
    print(f"state = {state}")


    # Load the metadata
    while True:
        try:
            md = metadata_loader(True, True)
            break
        except:
            sleep(5)
    md = md[md["job_status"] == "JobStatus.DONE"]
    md = md[md["code"] == "RepetitionCodeCircuit"]
    md = md[md["descr"] == 'subset RepCodes']
    md = md.dropna(subset=["rounds"])
    md = md[md["meas_level"] == 1]
    md['rounds'] = md['rounds'].astype(int)
    md['distance'] = md['distance'].astype(int)

    md = md[md["backend_name"] == DEVICE]
    md = md[md["logical"] == LOGICAL]
    md = md[md["xbasis"] == XBASIS]
    md = md[md["rounds"] == ROUNDS]

    # md = md[:1]
    print("shape:", md.shape)

    DOUBLE_MSMT = False # to get also 03-16

    jobs_by_calibration_date = {}
    for index, row in md.iterrows():
        job_id = row['job_id']

        while True:
            try:
                _, _, calib_creation_date = find_closest_calib_jobs(tobecalib_job=job_id, verbose=False, double_msmt=DOUBLE_MSMT)
                break
            except:
                sleep(5)

        if calib_creation_date not in jobs_by_calibration_date.keys():
            jobs_by_calibration_date[calib_creation_date] = [job_id]
        else:
            jobs_by_calibration_date[calib_creation_date].append(job_id)

    print(jobs_by_calibration_date)
    print()
    print(f"num of calibrations: {len(jobs_by_calibration_date)}")
    print(f"num of jobs per calibration: {([len(jobs) for jobs in jobs_by_calibration_date.values()])}")


    d = md["distance"].values[0]

    distances = np.arange(7, d+1, 4)
    distances = distances[::-1]

    
    HANDLE_OUTLIERS = True

    rel_error = 1
    _RESETS = False

    # KDE BANDWIDTHS
    lin = [0.6, 1.2, 1]
    num_points = 20
    # lin = [0.1, 0.7, 1]
    # num_points = 7
    bandwidths = np.linspace(lin[0], lin[1], lin[2])


    # Load or initialize the data JSON
    if not os.path.exists(file_name):
        data = {}
    else:
        with open(file_name, "r") as f:
            data = json.load(f)

    for calib_date in jobs_by_calibration_date.keys():
        job_ids = jobs_by_calibration_date[calib_date]
        num_jobs = len(job_ids)
        # get the noise dict of that date
        noise_dict = get_noise_dict_from_backend(provider, DEVICE, date = calib_date)

        # get the KDE of that date

        while True:
            try:
                all_memories, _, _ = load_calibration_memory(provider, tobecalib_backend=DEVICE, 
                                                                            other_date=calib_date, post_process=True,
                                                                            double_msmt=False)
                break
            except:
                sleep(5)

        kde_dict = csi.get_KDEs(all_memories, bandwidths, relError=rel_error, absError=-1, num_points=num_points)

        while True:
            try:
                _, _, msmt_err_dict_PS = load_calibration_memory(provider, tobecalib_backend=DEVICE, 
                                                                                other_date=calib_date, post_process=True,
                                                                                double_msmt=True)
                break
            except:
                sleep(5)
        # kde_dict_PS = csi.get_KDEs(all_memories_PS, bandwidths, relError=rel_error, absError=-1, num_points=num_points)

        # Get the mean msmt errors
        p_soft_mean = 0
        p_hard_mean = 0
        for key, value in msmt_err_dict_PS.items():
            p_soft_mean += value['p_soft']
            p_hard_mean += value['p_hard']
        p_soft_mean /= len(msmt_err_dict_PS)
        p_hard_mean /= len(msmt_err_dict_PS)
        print(msmt_err_dict_PS, p_soft_mean, p_hard_mean)


        # Retrieve the memories
        memories = []
        for job_id in tqdm(jobs_by_calibration_date[calib_date], desc=f"Retrieving jobs of {calib_date} calibration"):
            d = md[md["job_id"] == job_id]["distance"].values[0] 
            T = md[md["job_id"] == job_id]["rounds"].values[0] # Should be 10

            # Get the job
            job = provider.retrieve_job(job_id)
            memory = job.result().get_memory()
            nb_shots_per_job = len(memory)
            memories.append(memory)

        # Stack the memories vertically
        big_memory = np.vstack(memories)

        # Get the layout of the last job (same as previous)
        layout_des = job.deserialize_layout(job.initial_layouts()[0]) # only 1 layout
        link_qubits = list(layout_des['link_qubit'].values())
        code_qubits = list(layout_des['code_qubit'].values())

        # Get the pSoft and countMat matrices
        big_layout = link_qubits + code_qubits
        inverted_q_map = inv_qubit_mapping(get_repcode_IQ_map(big_layout, synd_rounds=T))

        print(f"Starting to get pSoft and countMat at {datetime.now()}")
        pSoft, countMat = csi.iqConvertor(big_memory, inverted_q_map, kde_dict, rel_error, -1,
                                                    handleOutliers = HANDLE_OUTLIERS)

        # Get overall layout and noise model
        layout = link_qubits + code_qubits
        avgs = get_avgs_from_dict(noise_dict, layout)
        noise_list = [avgs["two_gate"], avgs["single_gate"], avgs["t1_err"], avgs["t2_err"]]
        readout = avgs["readout"]
        noise_list += [(p_hard_mean+p_soft_mean), p_hard_mean, p_soft_mean]

        pSoft_mean = np.mean(pSoft)

        
        # Subsample decoding
        for D_NEW in tqdm(distances):
            subsets = generate_subsets_with_center(d, D_NEW)
            num_subsets = len(subsets)

            pSoft_big = np.vstack([pSoft[:, get_cols_to_keep(subset, T, d)] for subset in subsets])
            countMat_big = np.vstack([countMat[:, get_cols_to_keep(subset, T, d)] for subset in subsets])
            NB_SHOTS = countMat_big.shape[0]


            # Get stim models
            code_mean_for_soft = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                            subsampling=False, no_fin_soft=True, layout=None,
                                                            msmt_err_dict=None)
            model_mean_for_soft = code_mean_for_soft.circuits[LOGICAL].detector_error_model()

            code_mean_for_hard = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=noise_list,
                                                            subsampling=False, no_fin_soft=False, layout=None,
                                                            msmt_err_dict=None)
            model_mean_for_hard = code_mean_for_hard.circuits[LOGICAL].detector_error_model()

            new_noise_list = noise_list.copy()
            new_noise_list[-1] = pSoft_mean
            code_mean_mean = RepetitionCodeStimCircuit(D_NEW, T, XBASIS, _RESETS, noise_list=new_noise_list,
                                                            subsampling=False, no_fin_soft=False, layout=None,
                                                            msmt_err_dict=None)
            model_mean_mean = code_mean_mean.circuits[LOGICAL].detector_error_model()

            # Decoding
            res_s_K_mean = csi.decodeConvertorAll(model_mean_for_soft, countMat_big, pSoft_big, T, 
                                                int(LOGICAL), _RESETS, decode_hard=False)
            res_h_K_mean = csi.decodeConvertorAll(model_mean_for_hard, countMat_big, pSoft_big, T,
                                                int(LOGICAL), _RESETS, decode_hard=True)
            res_h_K_mean_mean = csi.decodeConvertorAll(model_mean_mean, countMat_big, pSoft_big, T,
                                                int(LOGICAL), _RESETS, decode_hard=True)

            # Initialize structure to store errors per job per subset for current D_NEW
            num_errs_by_job_and_method = {
                method: {job_id: [0] * num_subsets for job_id in job_ids}
                for method in ["soft_KDE", "hard_KDE", "hard_KDE_mean_pSoft"]
                }

            # Update the num_errs_by_job_and_method based on decoding results
            for method, result in [
                ("soft_KDE", res_s_K_mean), 
                ("hard_KDE", res_h_K_mean), 
                ("hard_KDE_mean_pSoft", res_h_K_mean_mean)
            ]:
                for err_idx in result.indices:
                    subset_idx = err_idx // (nb_shots_per_job * num_jobs)  # Corrected formula for subset index calculation
                    job_idx = (err_idx % (nb_shots_per_job * num_jobs)) // nb_shots_per_job
                    job_id = jobs_by_calibration_date[calib_date][job_idx]
                    num_errs_by_job_and_method[method][job_id][subset_idx] += 1


            # Ensure each job_id has the required structure in the data dictionary
            for job_id in job_ids:
                if job_id not in data:
                    data[job_id] = {"additional_info": {
                        "p_soft": p_soft_mean,
                        "p_hard": p_hard_mean,
                        "noise_list": noise_list,
                        "calib_readout": readout,
                        "pSoft_mean": pSoft_mean,
                    }, "distances": {}}

                if str(D_NEW) not in data[job_id]["distances"]:
                    data[job_id]["distances"][str(D_NEW)] = {"tot_shots": NB_SHOTS}

                for method, errs_by_subset in num_errs_by_job_and_method.items():
                    data[job_id]["distances"][str(D_NEW)][method] = {
                        "mean_errs": np.mean(errs_by_subset[job_id]) if errs_by_subset[job_id] else 0,  # Calculating mean errs if errs exist
                        "errs": errs_by_subset[job_id]  # Storing the raw error counts for each subset
                    }


    # Save the updated data structure to a file
    with open(file_name, "w") as f:
        json.dump(data, f, indent=4)

In [6]:
logicals = ['0', '1']
xbasis = [True, False]
rounds = [50]
devices = ["ibm_sherbrooke"]

for DEVICE in devices:
    for LOGICAL in logicals:
        for XBASIS in xbasis:
            for ROUNDS in rounds:
                state = "X" if XBASIS else "Z"
                state += LOGICAL

                file_name  = f'./results/result_day3/{DEVICE}_{state}_{ROUNDS}.json'

                decode(DEVICE, LOGICAL, XBASIS, ROUNDS, file_name)

state = X0
shape: (40, 18)
{datetime.datetime(2024, 3, 23, 13, 12, 1, 375000, tzinfo=datetime.timezone.utc): ['cqzda58czq6g0081he8g', 'cqzda38dvs8g008j69vg', 'cqzda1g8gdp0008fvt4g', 'cqzd9zqdvs8g008j69t0', 'cqzd9xzs9z7g008dpajg', 'cqzd9wfczq6g0081he80', 'cqzd9tfktf3g00883pdg', 'cqzd9rf8gdp0008fvt30', 'cqzd9pys9z7g008dpahg', 'cqzd9mpdvs8g008j69sg', 'cqzd9k68gdp0008fvt2g', 'cqzd9he8gdp0008fvt20'], datetime.datetime(2024, 3, 23, 13, 8, 53, 174000, tzinfo=datetime.timezone.utc): ['cqzd9e5k5z70008j0880', 'cqzd9addvs8g008j69s0', 'cqzd98d8gdp0008fvt1g', 'cqzd96ck5z70008j087g', 'cqzd92c8gdp0008fvt0g', 'cqzd904ktf3g00883pd0', 'cqzd8y3dvs8g008j69r0', 'cqzd8wbs9z7g008dpag0'], datetime.datetime(2024, 3, 16, 10, 33, 33, 406000, tzinfo=datetime.timezone.utc): ['cqtqaxv4x0mg008aj1w0', 'cqtqawbtxzj0008y0260', 'cqtqatvtxzj0008y025g', 'cqtqasb4x0mg008aj1vg', 'cqtqaqj88ev0008130eg', 'cqtqapapkcdg008e3dhg', 'cqtqama36d60008hzs70', 'cqtqajj4x0mg008aj1v0', 'cqtqah236d60008hzs6g', 'cqtqafstxzj0008y0250'], da

Retrieving jobs of 2024-03-23 13:12:01.375000+00:00 calibration: 100%|██████████| 12/12 [00:31<00:00,  2.61s/it]


Starting to get pSoft and countMat at 2024-04-02 10:57:02.011493


100%|██████████| 12/12 [04:21<00:00, 21.82s/it]


11:05:43 Warning: Z0 decoding. Negative T2 error -0.08 % for qubit 38, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
11:05:43 Warning: Z0 decoding. Negative T2 error -0.01 % for qubit 39, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
11:05:43 Warning: Z0 decoding. Negative T2 error -0.01 % for qubit 111, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
Found jobs for backend ibm_sherbrooke with closest execution date 2024-03-23 13:11:56.380893+00:00.
Found jobs for backend ibm_sherbrooke with closest execution date 2024-03-23 13:12:25.263713+00:00.
{0: {'p_hard': 0.005111760223520447, 'p_soft': 0.000476250952501905}, 1: {'p_hard': 0.00444500889001778, 'p_soft': 0.0052387

...

11:13:06 Warning: Z0 decoding. Negative T2 error -0.06 % for qubit 123, setting to 0.. IN FILE: /Users/mha/My_Drive/Desktop/Studium/Physik/MSc/Semester_3/IBM/IBM_GIT/Soft-Info/src/soft_info/Hardware/backend_noise.py, LINE: 52
Found jobs for backend ibm_sherbrooke with closest execution date 2024-03-16 10:11:14.156336+00:00.
Found jobs for backend ibm_sherbrooke with closest execution date 2024-03-23 09:05:45.817173+00:00.
{0: {'p_hard': 0.004667259334518669, 'p_soft': 6.3500127000254e-05}, 1: {'p_hard': 0.012192024384048768, 'p_soft': 0.004318008636017272}, 2: {'p_hard': 0.014319278638557276, 'p_soft': 0.011271272542545086}, 3: {'p_hard': 0.010922021844043688, 'p_soft': 0.001079502159004318}, 4: {'p_hard': 0.007112014224028448, 'p_soft': 0.0024765049530099063}, 5: {'p_hard': 0.010922021844043688, 'p_soft': 0.03321056642113284}, 6: {'p_hard': 0.019335788671577342, 'p_soft': 0.02136779273558547}, 7: {'p_hard': 0.006127762255524511, 'p_soft': 0.0015875031750063502}, 8: {'p_hard': 0.004921

Retrieving jobs of 2024-03-16 10:33:33.406000+00:00 calibration: 100%|██████████| 10/10 [00:25<00:00,  2.54s/it]


Starting to get pSoft and countMat at 2024-04-02 11:14:07.474902


 25%|██▌       | 3/12 [01:09<03:27, 23.10s/it]


KeyboardInterrupt: 

In [ ]:
logicals = ['0', '1']
xbasis = [True, False]
rounds = [75]
devices = ["ibm_sherbrooke"]

for DEVICE in devices:
    for LOGICAL in logicals:
        for XBASIS in xbasis:
            for ROUNDS in rounds:
                state = "X" if XBASIS else "Z"
                state += LOGICAL

                file_name  = f'./results/results_night2/{DEVICE}_{state}_{ROUNDS}.json'

                decode(DEVICE, LOGICAL, XBASIS, ROUNDS, file_name)

state = X0
shape: (40, 18)


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x1075ca9d0>>
Traceback (most recent call last):
  File "/Users/mha/.local/share/virtualenvs/Soft-Info-fMUpUe5a/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 
